# Restore a character-level sequence to sequence model from to generate predictions.

Source:
https://keras.io/examples/lstm_seq2seq_restore/

This script loads the s2s.h5 model saved by lstm_seq2seq.py and generates sequences from it. It assumes that no changes have been made (for example: latent_dim is unchanged, and the input data and model architecture are unchanged).

### Data download

Lots of neat sentence pairs datasets. http://www.manythings.org/anki/

In [1]:
from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import Input
import numpy as np
import re
import string
from string import digits

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'lang_pairs/pol.txt'

Using TensorFlow backend.


Let's read the sentences (input = English, target = Polish),
and extract the character sets for each language.

Target sequences get additional "start sequence" and "end sequence" characters.

In [2]:
# Vectorize the data.  We use the same approach as the training script.
# NOTE: the data must be identical, in order for the character -> integer
# mappings to be consistent.
# We omit encoding target_texts since they are not needed.
input_texts_en = []
target_texts_pl = []
all_eng_words = set()
all_pl_words = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    lines = lines[:1000]
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, licence = line.split('\t')
    
    input_text = input_text.lower()
    target_text = target_text.lower()
    
    input_text = re.sub("'", '', input_text)
    target_text = re.sub("'", '', target_text)
    input_text = re.sub(",", ' COMMA', input_text)
    target_text = re.sub(",", ' COMMA', target_text)
    
    exclude = set(string.punctuation)
    input_text = ''.join(ch for ch in input_text if ch not in exclude)
    target_text = ''.join(ch for ch in target_text if ch not in exclude)
    
    remove_digits = str.maketrans('', '', digits)
    input_text = input_text.translate(remove_digits)
    target_text = target_text.translate(remove_digits)
    
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = 'START_' + target_text + '_END'
    input_texts_en.append(input_text)
    target_texts_pl.append(target_text)
    split_input_words = input_text.split()
    split_target_words = target_text.split()
    for word in split_input_words:
        if word not in all_eng_words:
            all_eng_words.add(word)
    for word in split_target_words:
        if word not in all_pl_words:
            all_pl_words.add(word)

input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_pl_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_pl_words)
max_encoder_seq_length = max([len(txt) for txt in input_words])
max_decoder_seq_length = max([len(txt) for txt in target_words])

In [3]:
print('Number of samples:', len(input_texts_en))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
print(len(input_words))
print(len(target_words))
print(len(input_texts_en)*16*num_decoder_tokens)

Number of samples: 999
Number of unique input tokens: 583
Number of unique output tokens: 992
Max sequence length for inputs: 9
Max sequence length for outputs: 22
583
992
15856128


In [33]:
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])
encoder_input_data = np.zeros(
    (len(input_texts_en), max_encoder_seq_length), dtype='float32')
decoder_input_data = np.zeros((len(target_texts_pl), max_decoder_seq_length), dtype='float32')
decoder_target_data = np.zeros((len(target_texts_pl), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
for i, (input_text, target_text) in enumerate(zip(input_texts_en, target_texts_pl)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [5]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

embedding_size = 50

encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None,))
dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     29150       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     49600       input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [7]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=20,
          validation_split=0.05)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 949 samples, validate on 50 samples
Epoch 1/20





949/949 [==============================] - 11s 11ms/step - loss: 0.3579 - acc: 0.0635 - val_loss: 0.3309 - val_acc: 9.0909e-04
Epoch 2/20
949/949 [==============================] - 4s 4ms/step - loss: 0.3477 - acc: 0.0037 - val_loss: 0.3221 - val_acc: 9.0909e-04
Epoch 3/20
949/949 [==============================] - 5s 6ms/step - loss: 0.3212 - acc: 0.0037 - val_loss: 0.3209 - val_acc: 9.0909e-04
Epoch 4/20
949/949 [==============================] - 5s 5ms/step - loss: 0.3070 - acc: 0.0037 - val_loss: 0.3271 - val_acc: 9.0909e-04
Epoch 5/20
949/949 [==============================] - 6s 6ms/step - loss: 0.3016 - acc: 0.0037 - val_loss: 0.3337 - val_acc: 9.0909e-04
Epoch 6/20
949/949 [==============================] - 5s 6ms/step - loss: 0.2995 - acc: 0.0037 - val_loss: 0.3388 - val_acc: 9.0909e-04
Epoch 7/20
949/949 [===============

In [8]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          29150     
_________________________________________________________________
lstm_1 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 49,350
Trainable params: 49,350
Non-trainable params: 0
_________________________________________________________________


In [9]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [34]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [35]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index+1000: seq_index + 1001]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts_en[seq_index+1000])
    print('Decoded sentence:', decoded_sentence)

KeyError: '\t'